In [ ]:
### import urllib.request
from bs4 import BeautifulSoup
 
print('Beginning file download with urllib2...')
 
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=157297'
req = urllib.request.Request(url)
res = urllib.request.urlopen(url).read()
 
soup = BeautifulSoup(res,'html.parser')
soup = soup.find("div",class_="poster")
#img의 경로를 받아온다
imgUrl = soup.find("img")["src"]
 
#urlretrieve는 다운로드 함수
#img.alt는 이미지 대체 텍스트 == 마약왕
urllib.request.urlretrieve(imgUrl, soup.find("img")["alt"]+'.jpg')

In [126]:
import urllib.request
from bs4 import BeautifulSoup
from urllib.parse import quote
import re
import pandas as pd


# 만개의 레시피에서 조리법 크롤링
def get_recipe_site1(keyword) :
    keyword = quote(keyword) # 한글 사용
    
    site_url = "http://www.10000recipe.com"
    
    # 1. 사이트에서 음식 검색
    search_url = site_url + "/recipe/list.html?q={keyword}&order=reco&page=1".format(**{"keyword" : keyword})
    
    res = urllib.request.urlopen(search_url).read().decode("utf-8")
    bsp = BeautifulSoup(res, "lxml")
    
    
    # 2. 검색된 목록에서 제일 첫번째 조리법 선택(추천순으로 되어있음)
    first_food_detail_element = bsp.find("a", {"class":"thumbnail"})
    
    save_step = ""
    
    if first_food_detail_element != None and len(first_food_detail_element) > 0:
        first_food_url = site_url + first_food_detail_element["href"] # 검색한 음식의 첫번째 조리법(추천순)

        res = urllib.request.urlopen(first_food_url).read().decode("utf-8")
        bsp = BeautifulSoup(res, "lxml")

        # 조리법 글자만 추출
        step_list = bsp.find_all("div", {"id" : re.compile("^stepdescr")})

        for step in step_list:
            save_step += step.text + "@"
        
    return save_step
    
    
# 실행
# TODO : food_categories는 실제 데이터로 변경할 것
food_categories = ["제육볶음", "ㅁㄴㅇㄴ"] # 분류가능한 음식
food_recipe = {}

# 분류되어있는 음식별로 조리법 크롤링
for food_name in food_categories:
    recipe_text = get_recipe_site1(food_name) # 음식명
    food_recipe[food_name] = recipe_text      # 조리법 text

# 음식명, 조리법 데이터 저장
food_recipe_to_list = {"recipe_name" : list(food_recipe.keys()), "recipe_text" : list(food_recipe.values())}
df = pd.DataFrame(food_recipe_to_list)

# TODO : food_recipe DB 저장
# 임시로 csv에 저장
df.to_csv("./recipe/recipe.csv",
          mode="w",
          header=False,
          index=False,
          encoding="euc-kr")



In [122]:
print(food_recipe.values())

dict_values(['재료를 준비해주세요~.@양념재료를 모두 넣어주세요~.@양념재료가 잘 섞이도록해주세요~.@대파와 청양고추는 어슷썰어주고 양파는 1cm 두께로 썰어주세요~.@돼지고기는 한입크기로 썰어주세요~.@만들어둔 양념장을 고기에 넣어 버무려주세요~.@양념 후 바로 볶아도 되지만 냉장고에 30분정도 두어 숙성시켜주면 양념이 베어 더욱 맛있답니다~!!@팬에 식용유 2큰술과 대파를 넣고 강불로 3분정도 볶아 파기름을 내주세요~.@양념한 고기를 넣어주세요~.@중불로 볶아 고기를 완전히 익혀주세요~.\n약 8분정도면 다 익는답니다~!!@양파와 청양고추를 넣어주세요~.@강불로 2분정도 볶아준뒤 불을 꺼주세요~.@마지막으로 통깨를 뿌려주면 맛있는 제육볶음 완성입니다~!!@', '김치는 1/4포기를 준비해 먹기 좋게 썰어주고@대파는 송송 썰고, 양파는 채 썰어주세요.\n매콤하게 드시려면 청양고추를 함께 넣어도 된답니다.@팬에 참기름 1을 두르고 썰어 놓은 김치를 넣고\n달달 볶아주세요.@김치가 숨이 죽고 익기 시작하면\n꽁치통조림 1캔을 모두 넣어주세요@이때 국물까지 모조리 넣어주는 게 나름 비법이랍니다.\n국물 때문인지 간도 좋고\n감칠맛도 생기더라고요.@그러고 나서 통조림 캔들 이용해 1캔 물을 계량해\n넣어주세요.\n김치 염도에 따라 물의 양이 달라질 수 있으니\n참고하시고, 저는 1캔만 부어주었답니다.@그리고 설탕 1,다진 마늘 0.5, 된장 0.3을 넣어 주세요.\n설탕은 김치 신맛을 줄여주는 역할을 하니\n김치 익힘에 따라 조절해주시고\n된장은 비린 맛을 잡아 주는 담당을 한답니다.\n된장도 염도가 있으니\n간에 따라 양을 조절해 주신 센스!@보글보글 찌개가 끓기 시작하면\n미리 썰어둔 양피를 모두 넣어주세요.@그리고 고춧가루 2를 넣어 줍니다.@마지막에 대파를 넣어 한소끔 더 끓여 마무리해주세요.@백종원 꽁치김치찌개 완성 ^^@'])


In [ ]:
download_image()